In [1]:
(start-swank 4006)


;; Swank started at port: 4006.


T

In [2]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber18/dat/leap/cmd/")
(add-path "~/Development/amber18/dat/leap/lib/")
(add-path "~/Development/amber18/dat/leap/parm/")
(add-path "~/Development/amber18/dat/leap/prep/")
(add-path "~/Development/amber18/dat/antechamber/")

(#P"/Users/meister/Development/amber18/dat/antechamber/"
 #P"/Users/meister/Development/amber18/dat/leap/prep/"
 #P"/Users/meister/Development/amber18/dat/leap/parm/"
 #P"/Users/meister/Development/amber18/dat/leap/lib/"
 #P"/Users/meister/Development/amber18/dat/leap/cmd/"
 #P"/Users/meister/Development/cando/extensions/cando/src/tests/addions/")

In [3]:
(defparameter *chemleap* (cando:load-mol2 "alanine-leap.mol2"))

*CHEMLEAP*

In [4]:
(leap.core:clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

(#<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [5]:
(source "leaprc.water.tip3p")


add-atom-types doesn't do anything


T

In [6]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ions234lm_126_tip3p>
 #<FORCE-FIELD frcmod.ionsjc_tip3p> #<FORCE-FIELD frcmod.tip3p>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [7]:
(:= viewer (show *chemleap*)) viewer

#<NGLV:NGLWIDGET>

In [8]:
(nglv::add-representation viewer "spacefill")

T

In [ ]:
(leap.add-ions:add-ions *chemleap* :|Na+| 3)

In [12]:
(cando::show-aggregate *chemleap*)

A Jupyter Widget

In [13]:
(leap.add-ions:add-ions *chemleap* :|Cl-| 2)

Total charge -0.000000000000000027755575615628914
Adding 2 counter ions to ACE using 1A grid. 
Placed Cl- in ACE at 10.473259d0 7.987d0 -0.14420199999999994d0
Placed Cl- in ACE at -2.5267409999999995d0 1.987d0 -0.14420199999999994d0


NIL

In [14]:
(leap.add-ions:add-ions *chemleap* :|Na+| 3)

Total charge -2.0
Adding 3 counter ions to ACE using 1A grid. 
Placed Na+ in ACE at -3.408740999999999d0 5.118d0 1.9867980000000003d0
Placed Na+ in ACE at 13.591259d0 10.118d0 0.9867980000000003d0
Placed Na+ in ACE at -2.408740999999999d0 -1.8819999999999997d0 -1.0132019999999997d0


NIL

In [8]:
(defparameter *chemwater* (cando:load-mol2 "alanine-water.mol2"))


*CHEMWATER*

In [18]:
(:= viewer (show *chemwater*)) viewer

A Jupyter Widget

In [19]:
(leap.add-ions:add-ions *chemwater* :|Na+| 3 :|Cl-| 1)

Total charge -0.0009999999999997788
Adding 4 counter ions to ACE using 1A grid. 
Placed Na+ in ACE at -1.7402999999999977d0 13.136699999999998d0 12.396700000000003d0
Placed Cl- in ACE at -1.7402999999999977d0 13.136699999999998d0 16.396700000000003d0
Placed Na+ in ACE at 0.25970000000000226d0 10.136699999999998d0 14.396700000000003d0
Placed Na+ in ACE at -13.740299999999998d0 0.1366999999999976d0 13.396700000000003d0


NIL

In [20]:
(leap.add-ions:add-ions *chemwater* :|Cl-| 0)

Total charge 1.999
2 Cl- ions required to neutraize. 
Adding 2 counter ions to ACE using 1A grid. 
Placed Cl- in ACE at 0.23370000000000246d0 6.2479999999999976d0 14.508000000000003d0
Placed Cl- in ACE at -14.766299999999998d0 2.2479999999999976d0 16.508000000000003d0


NIL

In [29]:
(chem:VDW-RADIUS-FOR-ELEMENT :|Na|)

1.6d0

In [10]:
(defparameter *oleap* (core:make-cxx-object 'chem:octree))

*OLEAP*

In [11]:
(chem:oct-oct-tree-create *oleap* *chemleap* 1 1.0 2.51 4.0 0 t)

NIL

In [23]:
(multiple-value-bind (min-charge-point max-charge-point) (chem:oct-tree-init-charges *oleap*  1 1 2.51)
    (defparameter *max-charge-point* max-charge-point))

*MAX-CHARGE-POINT*

In [24]:
*max-charge-point*

[ 10.4763 7.99 -0.141202 ]

In [25]:
(chem:oct-tree-delete-sphere *oleap* *max-charge-point* 5.026)

NIL

In [15]:
(multiple-value-bind (min-charge-point max-charge-point) (chem:oct-tree-update-charge *oleap* *max-charge-point* -1.0 2.51)
    (defparameter *max-charge-point* max-charge-point))

*MAX-CHARGE-POINT*

In [16]:
*max-charge-point*

[ -2.52374 1.99 -0.141202 ]

In [15]:
(defparameter *oleap* (core:make-cxx-object 'chem:octree))

*OLEAP*

In [16]:
(chem:oct-oct-tree-create *oleap* *chemleap* 1 1.0 1.37 4.0 2.51 0 t)

NIL

In [17]:
(multiple-value-bind (min-charge-point max-charge-point) (chem:oct-tree-init-charges *oleap*  1 1 1.37)
    (defparameter *min-charge-point* min-charge-point))

*MIN-CHARGE-POINT*

In [18]:
*min-charge-point*

[ -2.40674 -0.88 -0.011202 ]

In [35]:
(chem:oct-tree-delete-sphere *oleap* *min-charge-point* 2.74)

NIL

In [194]:
(multiple-value-bind (min-charge-point max-charge-point) (chem:oct-tree-update-charge *oleap* *min-charge-point* 1.0 1.37)
    (defparameter *min-charge-point* min-charge-point))

*MIN-CHARGE-POINT*

In [195]:
*min-charge-point*

[ 1.61626 9.13 -0.001202 ]

In [21]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *a1* (chem:make-atom :Na :|Na|))
(chem:set-position *a1* *newposl*)
(chem:set-charge *a1* '1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *mol1* *res1*)
(defparameter *Naion* (chem:make-aggregate))
(chem:add-matter *Naion* *mol1*)

NIL

In [27]:
(chem:add-matter *chemleap* *Naion*)

NIL

In [13]:
(chem:vdw-radius-for-element :|Na|)

1.6d0

In [34]:
(chem:oct-tree-delete-sphere *oleap* *otl* *newposl* 1.5)

NIL

In [35]:
(defparameter *newposl2* (chem:oct-tree-update-charge *oleap* *otl* *newposl* 1.0 1.5))

*NEWPOSL2*

In [36]:
*newposl2*

[ 1.61626 9.13 -0.001202 ]

In [40]:
(defparameter *a1* (chem:make-atom :|Na+| :|Na+|))

An error occurred of type: TYPE-ERROR: 
  #<TYPE-ERROR>


NIL

In [42]:
(chem:get-name *a1*)

:|Na|

In [26]:
(defparameter *9lyzagg* (leap.pdb:load-pdb "9lyz.pdb"))

:ssbond for (SSBOND A 6 A 127)
:ssbond for (SSBOND A 30 A 115)
:ssbond for (SSBOND A 64 A 80)
:ssbond for (SSBOND A 76 A 94)
popped residue: #<PDB-RESIDUE LYS>
popped residue: #<PDB-RESIDUE VAL>
popped residue: #<PDB-RESIDUE PHE>
popped residue: #<PDB-RESIDUE GLY>
popped residue: #<PDB-RESIDUE ARG>
popped residue: #<PDB-RESIDUE CYS>
popped residue: #<PDB-RESIDUE GLU>
popped residue: #<PDB-RESIDUE LEU>
popped residue: #<PDB-RESIDUE ALA>
popped residue: #<PDB-RESIDUE ALA>
popped residue: #<PDB-RESIDUE ALA>
popped residue: #<PDB-RESIDUE MET>
popped residue: #<PDB-RESIDUE LYS>
popped residue: #<PDB-RESIDUE ARG>
popped residue: #<PDB-RESIDUE HIS>
popped residue: #<PDB-RESIDUE GLY>
popped residue: #<PDB-RESIDUE LEU>
popped residue: #<PDB-RESIDUE ASP>
popped residue: #<PDB-RESIDUE ASN>
popped residue: #<PDB-RESIDUE TYR>
popped residue: #<PDB-RESIDUE ARG>
popped residue: #<PDB-RESIDUE GLY>
popped residue: #<PDB-RESIDUE TYR>
popped residue: #<PDB-RESIDUE SER>
popped residue: #<PDB-RESIDUE LEU>


SIMPLE-WARNING: 
Could not immediately identify topology for (AMU . HEAD)
SIMPLE-WARNING: 
Could not identify topology for (AMU . HEAD)
SIMPLE-WARNING: 
Could not immediately identify topology for (NAG . MAIN)
SIMPLE-WARNING: 
Could not identify topology for (NAG . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (AMU . MAIN)
SIMPLE-WARNING: 
Could not identify topology for (AMU . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNING: 
Could not immediately identify topology for (HOH . MAIN)
SIMPLE-WARNI

*9LYZAGG*

In [27]:
(:= w (show *9lyzagg*)) w

A Jupyter Widget

In [31]:
 (defparameter *target-charge* 0)
(chem:map-atoms
     nil
     (lambda (r)
             (setf *target-charge* (+ (chem:get-charge r) *target-charge*)))
     *9lyzagg*)

NIL

In [32]:
*target-charge*

8.000000501750037d0

In [19]:
(defparameter *opdb* (core:make-cxx-object 'chem:octree))

*OPDB*

In [20]:
(defparameter *otpdb* (chem:oct-oct-tree-create *opdb* *9lyzagg* 1 1.0 1.37 4.0 0 t))

*OTPDB*

In [21]:
(defparameter *np1* (chem:oct-tree-init-charges *opdb* *otpdb* 1 1 1.5))

*NP1*

In [22]:
*np1*

[ 9.19465 24.121 15.9856 ]

In [23]:
(chem:oct-tree-delete-sphere *opdb* *otpdb* *np1* 2.32)

NIL

In [24]:
(defparameter *np2* (chem:oct-tree-update-charge *opdb* *otpdb* *np1* 1.0 1.16))

*NP2*

In [25]:
*np2*

[ -6.80535 6.121 18.9856 ]

In [10]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [11]:
(build-hydrogens *chemagg*)

NIL

In [12]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [13]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C_0    S


NIL

In [14]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [13]:
(cando:jostle *chemagg*)

NIL

In [14]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          16889.091
 minSDnP       0     2     -8.04   0.0        2595175.764          12036.749
 minSDnP       0     3     -8.75   0.0        2328731.215           5958.852
DONE absolute force test:
forceRmsMag(4513.820269).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     4      -inf   0.0              0.000           4513.820
 minCGnP       0     5     -7.89  37.4        1546179.728           3445.965
 minCGnP       0     6     -7.06  55.1        1205757.834           4417.259
 minCGnP       0     7     -8.17  47.5        1034977.584           2546.811
 minCGnP       0     8     -7.42  56.4         936334.529           3176.241
 minCGnP       0     9     -8.53  62.3         860287.559           4170.326
 minCGnP       0    10     -8.15  40.5   

*EE*

In [15]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [16]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [17]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *a1* (chem:make-atom :CL :|Cl|))
(chem:set-position *a1* '(0.0 0.0 0.0))
(chem:set-charge *a1* '-1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *mol1* *res1*)
(defparameter *clion* (chem:make-aggregate))
(chem:add-matter *clion* *mol1*)

NIL

In [18]:
(defparameter *o1* (core:make-cxx-object 'chem:octree))

*O1*

In [19]:
(defparameter *ot* (chem:oct-oct-tree-create *o1* *chemagg* 1 1.0 1.0 4.0 0 t))

*OT*

In [20]:
(defparameter *newpos* (chem:oct-tree-init-charges *o1* *ot* 1 1 1.5))

*NEWPOS*

In [21]:
*newpos*

[ 200.413 319.565 -2.55179 ]

In [21]:
(defparameter *clion-copy* (chem:matter-copy *clion*))

*CLION-COPY*

In [22]:
(defparameter *clion-transform* (geom:make-m4-translate *newpos*))

*CLION-TRANSFORM*

In [23]:
(chem:apply-transform-to-atoms *clion-copy* *clion-transform*)

NIL

In [24]:
(chem:add-matter *chemagg* *clion-copy*)

NIL

In [25]:
(defparameter *ot* (chem:oct-oct-tree-create *o1* *chemagg* 1 1.0 1.0 4.0 0 t))

*OT*

In [26]:
(defparameter *newpos1* (chem:oct-tree-init-charges *o1* *ot* 1 1 1.5))

*NEWPOS1*

In [ ]:
(chem:oct-tree-delete-sphere *o1* *ot* *newpos* 1.5)

In [ ]:
(chem:oct-tree-update-charge *o1* *ot* *newpos* -1.0 1.5)

In [11]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/addions/alanine.mol2


NIL

In [12]:
(defparameter *chemagg2* (chem:matter-copy *chemagg*))

*CHEMAGG2*

In [13]:
(defparameter *chemcenter* (chem:geometric-center *chemagg2*))

*CHEMCENTER*

In [14]:
(defparameter *transform* (geom:make-m4-translate (geom:v* *chemcenter* -1.0)))

*TRANSFORM*

In [19]:
(chem:apply-transform-to-atoms *chemagg2* *transform*))

NIL

In [10]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [17]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [18]:
(nglv::show-aggregate *solvent*)

A Jupyter Widget

In [19]:
(assign-atom-types *chemleap*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemleap*)

NIL

In [20]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemleap* *solvent* '(10.0 10.0 10.0)))

Total bounding box for atom centers:  25.250742000000002d0 27.647735d0 23.02103d0


*NEWAGG*

In [21]:
(nglv::show-aggregate *newagg*)

A Jupyter Widget

In [29]:
(save-mol2 *newagg* "alanine-water.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/addions/alanine-water.mol2


NIL

In [21]:
(leap.add-ions:add-ions *newagg* :|Na+| 2)

Total charge 0.0
Adding 2 counter ions to ACE using 1A grid. 
Placed Na+ in ACE at -1.471889537811279d0 9.405495601654053d0 11.664794403076172d0
Placed Na+ in ACE at -2.471889537811279d0 -7.594504398345947d0 12.664794403076172d0


NIL

In [20]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

An error occurred of type: UNBOUND-VARIABLE: 
  #<UNBOUND-VARIABLE>


NIL

In [30]:
(chem:map-residues nil (lambda (r) (print r)) *newagg*)


#<RESIDUE :ACE> 
#<RESIDUE :ALA> 
#<RESIDUE :NME> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WA

NIL

In [29]:
(multiple-value-list (values))

NIL

In [11]:
(leap.off:load-off "/Users/tuj24515/Development/amber16/dat/leap/lib/atomic_ions.lib")

NIL

In [9]:
(defparameter *a1* (chem:make-atom :CL :|Cl-|))

An error occurred of type: TYPE-ERROR: 
  #<TYPE-ERROR>


NIL

In [10]:
(apropos "flags")

CLASP-FFI:%OWNERSHIP-FLAGS  Function
CORE:*BUILD-LINKFLAGS*  Constant
CFFI-TOOLCHAIN:*CC-FLAGS*  has value
CFFI-TOOLCHAIN:*LD-DLL-FLAGS*  has value
CFFI-TOOLCHAIN:*LD-EXE-FLAGS*  has value
CORE:ARRAY-FLAGS  Function
:CC-FLAGS  Constant
CFFI-GROVEL::CC-FLAGS
CFFI-GROVEL::CC-FLAGS-MIXIN
CFFI-GROVEL::CC-FLAGS-OF  Function
:CFLAGS  Constant
CFFI-TOOLCHAIN::CFLAGS
CORE::DEFAULT-LINK-FLAGS  Function
LLVM-SYS:DIFLAGS-APPLE-BLOCK
LLVM-SYS:DIFLAGS-ARTIFICIAL
LLVM-SYS:DIFLAGS-BIT-FIELD
LLVM-SYS:DIFLAGS-BLOCK-BYREF-STRUCT
LLVM-SYS:DIFLAGS-ENUM  has value
LLVM-SYS:DIFLAGS-EXPLICIT
LLVM-SYS:DIFLAGS-EXTERNAL-TYPE-REF
LLVM-SYS:DIFLAGS-FWD-DECL
LLVM-SYS:DIFLAGS-INTRODUCED-VIRTUAL
LLVM-SYS:DIFLAGS-LVALUE-REFERENCE
LLVM-SYS:DIFLAGS-MULTIPLE-INHERITANCE
LLVM-SYS:DIFLAGS-NO-RETURN
LLVM-SYS:DIFLAGS-OBJC-CLASS-COMPLETE
LLVM-SYS:DIFLAGS-OBJECT-POINTER
LLVM-SYS:DIFLAGS-PRIVATE
LLVM-SYS:DIFLAGS-PROTECTED
LLVM-SYS:DIFLAGS-PROTOTYPED
LLVM-SYS:DIFLAGS-PUBLIC
LLVM-SYS:DIFLAGS-RVALUE-REFERENCE
LLVM-SYS:DIFLAGS-SING

NIL

In [14]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *a1* (chem:make-atom :CL :|Cl|))
(chem:set-position *a1* '(0.0 0.0 0.0))
(chem:set-charge *a1* '-1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *mol1* *res1*)
(defparameter *clion* (chem:make-aggregate))
(chem:add-matter *clion* *mol1*)
(defparameter *clion-copy1* (chem:matter-copy *clion*))
(defparameter *clion-transform* (geom:make-m4-translate *newposl*))
(chem:apply-transform-to-atoms *clion-copy* *clion-transform*)
(chem:add-matter *chemleap* *clion-copy*)

SIMPLE-PROGRAM-ERROR: 
The symbol CANDO-USER::*NEWPOSL* is unbound


NIL

In [30]:
 (chem:get-net-charge *chemleap*)

SIMPLE-PROGRAM-ERROR: 
There are no applicable methods of GET-NET-CHARGE for receiver class AGGREGATE


NIL

In [30]:
 (chem:get-net-charge *chemleap*)

SIMPLE-PROGRAM-ERROR: 
There are no applicable methods of GET-NET-CHARGE for receiver class AGGREGATE


NIL

*NEWPOSL*

In [ ]:
(picked-atoms viewer)